In [33]:
import pandas as pd

## Data Cleaning

### Data Reading

In [56]:
population_data = pd.read_csv('raw_Population by municipality.csv')
life_expectancy_data = pd.read_csv('raw_Life expectancy by municipality.csv')
ahqi_data = pd.read_csv('raw_Air quality index by municipality.csv')
wildfire_data = pd.read_csv('cleaned_Alberta_wildfires_transformed.csv')
canola_data = pd.read_csv('cleaned_Canola_crops_by_municipality.csv')
cattle_data = pd.read_csv('cleaned_Cattle and calves by municipality.csv')
dwelling_data = pd.read_csv('cleaned_Dwelling_Units_by_Municipality.csv')
barley_data = pd.read_csv('cleaned_barley_crops_by_municipality.csv')
duram_wheat_data = pd.read_csv('cleaned_duramwheat_crops_by_municipality.csv')
housing_data = pd.read_csv('raw_housing_buildings_by_municipality.csv')
other_wheat_data = pd.read_csv('cleaned_non_duramwheat_crops_by_municipality.csv')
oat_crop_data = pd.read_csv('cleaned_oats_crops_by_municipality.csv')
other_crops_data = pd.read_csv('cleaned_othercrop_crops_by_municipality.csv')
pigs_data = pd.read_csv('cleaned_pigs by municipality.csv')
housing_starts_data = pd.read_csv('cleaned_total_housing starts by municipality.csv')
oil_wells_data = pd.read_csv('cleaned_total_well count by municipality.csv')
vehicle_data = pd.read_csv('cleaned_vehicle_registration.csv')
natural_gas_data = pd.read_csv('raw_NaturalGasproduction_bymunicipality.csv')
greenhouse_gas_data = pd.read_csv('raw_GreenhouseGasEmissions_bymunicipality.csv')

### Transformation

#### Air Quality Index

In [57]:
ahqi_data['Air Quality Health Index'] = ahqi_data['Air Quality Health Index'].replace("10+", 10).astype(int)
ahqi_data['Weighted_AQHI'] = ahqi_data['Air Quality Health Index'] * ahqi_data['OriginalValue']
weighted_aqhi = ahqi_data.groupby(['CSD', 'Period']).apply(
    lambda x: x['Weighted_AQHI'].sum() / x['OriginalValue'].sum()
).reset_index(name='Adjusted_AQHI')
weighted_aqhi.rename(columns={'OriginalValue': 'Air Quality Health Index'}, inplace=True)
weighted_aqhi["Key"] = weighted_aqhi["CSD"].astype(str) + weighted_aqhi["Period"].astype(str)
weighted_aqhi.head()

<ipython-input-57-e5a6308b91ff>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  weighted_aqhi = ahqi_data.groupby(['CSD', 'Period']).apply(


,CSD,Period,Adjusted_AQHI,Key
0,Airdrie,2017,2.348223,Airdrie2017
1,Airdrie,2018,2.617466,Airdrie2018
2,Airdrie,2019,2.251483,Airdrie2019
3,Airdrie,2020,2.163560,Airdrie2020
4,Airdrie,2021,2.361184,Airdrie2021


#### How does population size, life expectancy, influence air quality trends across municipalities?


In [58]:
population_combined = population_data.groupby(['CSD', 'Period', 'Age Group'], as_index=False)['Number of People'].sum()
population_pivot = population_combined.pivot(index=['CSD', 'Period'], columns='Age Group', values='Number of People')
population_pivot.reset_index(inplace=True)
population_pivot.rename(columns={'09-May': '05-09','14-Oct': '10-14'}, inplace=True)
population_pivot.columns = [
    f"{col} (Count of People)" if col not in ['CSD', 'Period'] else col for col in population_pivot.columns
]
population_pivot["Key"] = population_pivot["CSD"].astype(str) + population_pivot["Period"].astype(str)
population_pivot.head()

,CSD,Period,0-4 (Count of People),05-09 (Count of People),10-14 (Count of People),15-19 (Count of People),20-24 (Count of People),25-29 (Count of People),30-34 (Count of People),35-39 (Count of People),40-44 (Count of People),45-49 (Count of People),50-54 (Count of People),55-59 (Count of People),Key
0,Acadia No. 34,2001,42,9,37,39,38,27,30,45,37,39,32,26,Acadia No. 342001
1,Acadia No. 34,2002,42,7,40,38,39,28,30,39,41,39,34,30,Acadia No. 342002
2,Acadia No. 34,2003,45,8,42,28,41,32,30,34,42,38,36,32,Acadia No. 342003
3,Acadia No. 34,2004,38,10,46,31,34,36,33,34,39,38,37,32,Acadia No. 342004
4,Acadia No. 34,2005,32,7,51,34,39,28,33,35,40,37,39,32,Acadia No. 342005


In [59]:
life_expectancy_combined = life_expectancy_data.groupby(['CSD', 'Period'], as_index=False)['OriginalValue'].mean()
life_expectancy_combined.rename(columns={'OriginalValue': 'Life Expectancy (Years)'}, inplace=True)
life_expectancy_combined ["Key"] = life_expectancy_combined ["CSD"].astype(str) + life_expectancy_combined ["Period"].astype(str)
life_expectancy_combined.head()

,CSD,Period,Life Expectancy (Years),Key
0,Acadia No. 34,2011,80.106667,Acadia No. 342011
1,Acadia No. 34,2012,80.406667,Acadia No. 342012
2,Acadia No. 34,2013,81.070000,Acadia No. 342013
3,Acadia No. 34,2014,81.726667,Acadia No. 342014
4,Acadia No. 34,2015,81.976667,Acadia No. 342015


#### Does dwellings, construction, and vehicle usage have an impact on the air quality index?


In [60]:
housing_transformed = housing_data.copy()
housing_transformed.drop({"IndicatorSummaryDescription","UnitOfMeasure"},axis=1,inplace=True)
housing_pivot = housing_transformed.pivot(index=['CSD', 'Period'], columns='Structural Type', values='OriginalValue')
housing_pivot.rename(columns={'Apartment, building that has fewer than five storeys': 'Apartment, less than 5 stories', "Apartment, building that has five or more storeys" : "Apartment, 5 or more stories" }, inplace=True)
housing_pivot.columns = [
    f"{col} (Count)" if col not in ['CSD', 'Period'] else col for col in housing_pivot.columns
]
housing_pivot.reset_index(inplace=True)
housing_pivot.columns.name = None
housing_pivot["Key"] = housing_pivot["CSD"].astype(str) + housing_pivot["Period"].astype(str)
housing_pivot.head()

,CSD,Period,"Apartment, less than 5 stories (Count)","Apartment, 5 or more stories (Count)","Apartment, duplex (Count)",Movable dwelling (Count),Other attached dwelling (Count),Row house (Count),Semi-detached house (Count),Single-detached house (Count),Key
0,Acadia No. 34,2001,0,0,0,0,0,0,0,160,Acadia No. 342001
1,Acadia No. 34,2006,0,0,0,10,0,0,0,150,Acadia No. 342006
2,Acadia No. 34,2011,0,0,0,10,0,0,0,140,Acadia No. 342011
3,Acadia No. 34,2016,0,0,0,5,0,0,0,155,Acadia No. 342016
4,Acadia No. 34,2021,0,0,0,35,0,0,0,125,Acadia No. 342021


In [61]:
dwelling_transformed = dwelling_data.copy()
dwelling_transformed.rename(columns={'Total_Dwelling_Count': 'Dwelling Units (Count)'}, inplace=True)
dwelling_transformed["Key"] = dwelling_transformed["CSD"].astype(str) + dwelling_transformed["Period"].astype(str)
dwelling_transformed.head()

,CSD,Period,Dwelling Units (Count),Key
0,Sedgewick,1994,364.0,Sedgewick1994
1,Sedgewick,1995,378.0,Sedgewick1995
2,Sedgewick,1996,380.0,Sedgewick1996
3,Sedgewick,1997,384.0,Sedgewick1997
4,Sedgewick,1998,388.0,Sedgewick1998


In [62]:
housing_starts_transformed = housing_starts_data
housing_starts_transformed.rename(columns={'Total # of residence/housing starts': 'Residence/Housing Starts (Count)','Year': 'Period'}, inplace=True)
housing_starts_transformed["Key"] = housing_starts_transformed["CSD"].astype(str) + housing_starts_transformed["Period"].astype(str)
housing_starts_transformed.head()

,CSD,Period,Residence/Housing Starts (Count),Key
0,Red Deer,1987,490,Red Deer1987
1,Red Deer,1988,426,Red Deer1988
2,Red Deer,1989,402,Red Deer1989
3,Red Deer,1990,350,Red Deer1990
4,Red Deer,1991,294,Red Deer1991


In [63]:
vehicle_transformed = vehicle_data.copy()
vehicle_transformed.rename(columns={'Total_Vehicle_Registration_Count': 'Vehicle Registration (Count)'}, inplace=True)
vehicle_transformed["Key"] = vehicle_transformed["CSD"].astype(str) + vehicle_transformed["Period"].astype(str)
vehicle_transformed.drop("Vehicle_Type",axis=1,inplace=True)
vehicle_transformed.head()

,CSD,Period,Vehicle Registration (Count),Key
0,Sedgewick,2005,1743,Sedgewick2005
1,Sedgewick,2006,1733,Sedgewick2006
2,Sedgewick,2007,1805,Sedgewick2007
3,Sedgewick,2008,1827,Sedgewick2008
4,Sedgewick,2009,1846,Sedgewick2009


#### How does energy production affect the air quality index across Alberta?


In [64]:
natural_gas_transformed = natural_gas_data.copy()
natural_gas_transformed.drop("CSDUID",axis=1,inplace=True)
natural_gas_transformed["Key"] = natural_gas_transformed["CSD"].astype(str) + natural_gas_transformed["Period"].astype(str)
natural_gas_transformed.head()

,CSD,Period,Natural Gas Production (m^3),Key
0,Drumheller,2003,104493.2,Drumheller2003
1,Drumheller,2004,105486.4,Drumheller2004
2,Drumheller,2005,130930.0,Drumheller2005
3,Drumheller,2006,128564.0,Drumheller2006
4,Drumheller,2007,124354.0,Drumheller2007


In [65]:
oil_wells_transformed = oil_wells_data.copy()
oil_wells_transformed.rename(columns={'Well Count': 'Oil Wells (Count)','Year': 'Period'}, inplace=True)
oil_wells_transformed["Key"] = oil_wells_transformed["CSD"].astype(str) + oil_wells_transformed["Period"].astype(str)
oil_wells_transformed.head()

,CSD,Period,Oil Wells (Count),Key
0,Drumheller,2003,10,Drumheller2003
1,Drumheller,2004,35,Drumheller2004
2,Drumheller,2005,21,Drumheller2005
3,Drumheller,2006,17,Drumheller2006
4,Drumheller,2007,11,Drumheller2007


#### How do greenhouse gas emissions influence the air quality index across Alberta?


In [66]:
greenhouse_gas_transformed = greenhouse_gas_data.copy()
greenhouse_gas_transformed.drop("CSDUID",axis=1,inplace=True)
greenhouse_gas_transformed["Key"] = greenhouse_gas_transformed["CSD"].astype(str) + greenhouse_gas_transformed["Period"].astype(str)
greenhouse_gas_transformed.head()

,CSD,Period,CH4 Equivalent (t),CO2 Equivalent (t),N2O Equivalent (t),Key
0,Cypress County,2004,25774.24966,428901.0228,4106.440,Cypress County2004
1,Cypress County,2005,3822.50005,171155.5000,2652.200,Cypress County2005
2,Cypress County,2006,4104.99993,367230.4000,2235.000,Cypress County2006
3,Cypress County,2007,4544.00005,350541.8163,1666.118,Cypress County2007
4,Cypress County,2008,4611.50013,374139.2797,1752.240,Cypress County2008


#### How impactful is the agriculture industry in affecting air quality throughout Alberta?


##### Animal Data

In [67]:
cattle_transformed = cattle_data.copy()
cattle_transformed.rename(columns={'Count': 'Cattle and Calves (Count)'}, inplace=True)
cattle_transformed["Key"] = cattle_transformed["CSD"].astype(str) + cattle_transformed["Period"].astype(str)
cattle_transformed.drop("IndicatorSummaryDescription",axis=1,inplace=True)
cattle_transformed.head()

,CSD,Period,Cattle and Calves (Count),Key
0,Starland County,2001,0,Starland County2001
1,Starland County,2006,34341,Starland County2006
2,Starland County,2011,35178,Starland County2011
3,Starland County,2016,54027,Starland County2016
4,Starland County,2021,33028,Starland County2021


In [68]:
pigs_transformed = pigs_data.copy()
pigs_transformed.rename(columns={'Count': 'Pigs (Count)'}, inplace=True)
pigs_transformed["Key"] = pigs_transformed["CSD"].astype(str) + pigs_transformed["Period"].astype(str)
pigs_transformed.drop("IndicatorSummaryDescription",axis=1,inplace=True)
pigs_transformed.head()

,CSD,Period,Pigs (Count),Key
0,Starland County,2001,0,Starland County2001
1,Starland County,2006,31367,Starland County2006
2,Starland County,2011,29348,Starland County2011
3,Starland County,2016,58575,Starland County2016
4,Starland County,2021,35116,Starland County2021


##### Plant

In [69]:
other_crops_transformed = other_crops_data.copy()
other_crops_transformed.rename(columns={'Count': 'Other Crops (Count)'}, inplace=True)
other_crops_transformed["Key"] = other_crops_transformed["CSD"].astype(str) + other_crops_transformed["Period"].astype(str)
other_crops_transformed.drop("Crop Type",axis=1,inplace=True)
other_crops_transformed.head()

,CSD,Period,Other Crops (Count),Key
0,Starland County,2006,64335,Starland County2006
1,Starland County,2011,65355,Starland County2011
2,Starland County,2016,86197,Starland County2016
3,Starland County,2021,89193,Starland County2021
4,Kneehill County,2006,77079,Kneehill County2006


In [70]:
canola_transformed = canola_data.copy()
canola_transformed.rename(columns={'Count': 'Canola (Count)'}, inplace=True)
canola_transformed["Key"] = canola_transformed["CSD"].astype(str) + canola_transformed["Period"].astype(str)
canola_transformed.drop("Crop Type",axis=1,inplace=True)
canola_transformed.head()

,CSD,Period,Canola (Count),Key
0,Starland County,2006,69750,Starland County2006
1,Starland County,2011,105159,Starland County2011
2,Starland County,2016,116519,Starland County2016
3,Starland County,2021,111193,Starland County2021
4,Kneehill County,2006,133570,Kneehill County2006


In [71]:
barley_transformed = barley_data.copy()
barley_transformed.rename(columns={'Count': 'Barley (Count)'}, inplace=True)
barley_transformed["Key"] = barley_transformed["CSD"].astype(str) + barley_transformed["Period"].astype(str)
barley_transformed.drop("Crop Type",axis=1,inplace=True)
barley_transformed.head()

,CSD,Period,Barley (Count),Key
0,Starland County,2006,74158,Starland County2006
1,Starland County,2011,58694,Starland County2011
2,Starland County,2016,78553,Starland County2016
3,Starland County,2021,77908,Starland County2021
4,Kneehill County,2006,145640,Kneehill County2006


In [72]:
durum_wheat_transformed = duram_wheat_data.copy()
durum_wheat_transformed.rename(columns={'Count': 'Durum Wheat (Count)'}, inplace=True)
durum_wheat_transformed["Key"] = durum_wheat_transformed["CSD"].astype(str) + durum_wheat_transformed["Period"].astype(str)
durum_wheat_transformed.drop("Crop Type",axis=1,inplace=True)
durum_wheat_transformed.head()

,CSD,Period,Durum Wheat (Count),Key
0,Starland County,2006,1360,Starland County2006
1,Starland County,2011,0,Starland County2011
2,Starland County,2016,1380,Starland County2016
3,Starland County,2021,0,Starland County2021
4,Kneehill County,2006,4926,Kneehill County2006


In [73]:
other_wheat_transformed = other_wheat_data.copy()
other_wheat_transformed.rename(columns={'Count': 'Non-Durum Wheat (Count)'}, inplace=True)
other_wheat_transformed["Key"] = other_wheat_transformed["CSD"].astype(str) + other_wheat_transformed["Period"].astype(str)
other_wheat_transformed.drop("Crop Type",axis=1,inplace=True)
other_wheat_transformed.head()

,CSD,Period,Non-Durum Wheat (Count),Key
0,Starland County,2006,183625,Starland County2006
1,Starland County,2011,147156,Starland County2011
2,Starland County,2016,132256,Starland County2016
3,Starland County,2021,142320,Starland County2021
4,Kneehill County,2006,241344,Kneehill County2006


In [74]:
oat_crop_transformed = oat_crop_data.copy()
oat_crop_transformed.rename(columns={'Count': 'Oats (Count)'}, inplace=True)
oat_crop_transformed["Key"] = oat_crop_transformed["CSD"].astype(str) + oat_crop_transformed["Period"].astype(str)
oat_crop_transformed.drop("Crop Type",axis=1,inplace=True)
oat_crop_transformed.head()

,CSD,Period,Oats (Count),Key
0,Starland County,2006,8563,Starland County2006
1,Starland County,2011,3906,Starland County2011
2,Starland County,2016,3604,Starland County2016
3,Starland County,2021,3832,Starland County2021
4,Kneehill County,2006,9627,Kneehill County2006


#### Do seasonal wildfires significantly impact the average air quality index across Alberta?

In [75]:
wildfire_transformed = wildfire_data.copy()
wildfire_transformed.rename(columns={'Municipality': 'CSD','Year': 'Period',"Sum_firesize": "Fire Size (Hectares)", "Count_fires"	: "Fires (Count)"}, inplace=True)
wildfire_transformed["Key"] = wildfire_transformed["CSD"].astype(str) + wildfire_transformed["Period"].astype(str)
wildfire_transformed.head()

,CSD,Period,Fire Size (Hectares),Fires (Count),Key
0,Athabasca County,2016,10.1,2,Athabasca County2016
1,Big Lakes County,2017,12.4,1,Big Lakes County2017
2,Brazeau County,2013,14.7,3,Brazeau County2013
3,Census Division No. 9,2013,21.0,3,Census Division No. 92013
4,Clear Hills County,2017,21.9,5,Clear Hills County2017


## Masterdata addition

### Masterdata Creation

In [76]:
unique_csds_periods = population_data[['CSD', 'Period']].drop_duplicates()
unique_csds_periods = unique_csds_periods.reset_index(drop=True)
masterframe = unique_csds_periods

In [77]:
masterframe["Key"] = masterframe["CSD"].astype(str) + masterframe["Period"].astype(str)

### Masterdata Merging

In [78]:
finished_frame_list = [population_pivot,life_expectancy_combined,
                       oil_wells_transformed,natural_gas_transformed,
                       greenhouse_gas_transformed,
                       oat_crop_transformed,canola_transformed,barley_transformed,durum_wheat_transformed,other_wheat_transformed,other_crops_transformed,
                       cattle_transformed,pigs_transformed,
                       dwelling_transformed,housing_pivot,housing_starts_transformed,vehicle_transformed,
                       wildfire_transformed,weighted_aqhi]

In [79]:
for frame in finished_frame_list:
    columns_to_merge = ['Key'] + [col for col in frame.columns if col not in ['CSD', 'Period', 'Key']]
    frame_subset = frame[columns_to_merge]
    masterframe = masterframe.merge(frame_subset, on='Key', how='left')

In [80]:
masterframe.head()

,CSD,Period,Key,0-4 (Count of People),05-09 (Count of People),10-14 (Count of People),15-19 (Count of People),20-24 (Count of People),25-29 (Count of People),30-34 (Count of People),...,Movable dwelling (Count),Other attached dwelling (Count),Row house (Count),Semi-detached house (Count),Single-detached house (Count),Residence/Housing Starts (Count),Vehicle Registration (Count),Fire Size (Hectares),Fires (Count),Adjusted_AQHI
0,Sedgewick,2001,Sedgewick2001,40,10,45,92,59,45,49,...,5.0,5.0,5.0,5.0,320.0,NaN,NaN,NaN,NaN,NaN
1,Sedgewick,2002,Sedgewick2002,39,10,45,93,62,45,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Sedgewick,2003,Sedgewick2003,40,9,44,97,57,51,48,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sedgewick,2004,Sedgewick2004,41,7,42,98,65,46,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sedgewick,2005,Sedgewick2005,47,4,49,85,69,54,50,...,NaN,NaN,NaN,NaN,NaN,NaN,1743.0,NaN,NaN,NaN


## Masterdata missing value checks and to CSV


In [81]:
missing_values = masterframe.isna().sum()
print(missing_values)
print(masterframe.shape)

CSD                                          0
Period                                       0
Key                                          0
0-4 (Count of People)                        0
05-09 (Count of People)                      0
10-14 (Count of People)                      0
15-19 (Count of People)                      0
20-24 (Count of People)                      0
25-29 (Count of People)                      0
30-34 (Count of People)                      0
35-39 (Count of People)                      0
40-44 (Count of People)                      0
45-49 (Count of People)                      0
50-54 (Count of People)                      0
55-59 (Count of People)                      0
Life Expectancy (Years)                   5870
Oil Wells (Count)                         8128
Natural Gas Production (m^3)              7939
CH4 Equivalent (t)                        2134
CO2 Equivalent (t)                        2134
N2O Equivalent (t)                        2134
Oats (Count) 

## Subset Creation

In [82]:
ahqi_subset = masterframe[masterframe['Adjusted_AQHI'].notna()]
print(ahqi_subset.isna().sum())
print(ahqi_subset.shape)

CSD                                         0
Period                                      0
Key                                         0
0-4 (Count of People)                       0
05-09 (Count of People)                     0
10-14 (Count of People)                     0
15-19 (Count of People)                     0
20-24 (Count of People)                     0
25-29 (Count of People)                     0
30-34 (Count of People)                     0
35-39 (Count of People)                     0
40-44 (Count of People)                     0
45-49 (Count of People)                     0
50-54 (Count of People)                     0
55-59 (Count of People)                     0
Life Expectancy (Years)                    17
Oil Wells (Count)                         159
Natural Gas Production (m^3)              131
CH4 Equivalent (t)                         28
CO2 Equivalent (t)                         28
N2O Equivalent (t)                         28
Oats (Count)                      

## To CSV

In [83]:
masterframe.to_csv('masterdata.csv', index=False)

In [84]:
ahqi_subset.to_csv('cleaned_AHQI_from_Masterdata.csv', index=False)
